# A.7 Numba를 이용하여 빠른 NumPy 함수 작성하기

In [23]:
# Numba(눔바)는 CPU, GPU 또는 기타 하드웨어를 이용하여 NumPy와 유사한 데이터를 다루는 빠른 함수를 작성할 수 있도록 도와주는 오픈소스 프로젝트다.
# 파이썬 코드를 컴파일된 기계 코드로 변환하기 위해 LLVM 프로젝트를 사용하고 있다.

In [24]:
# Numba를 소개하기 위해 for 문을 사용하여 (x - y).mean()을 계산하는 순수 파이썬 코드를 작성하자.

In [25]:
import numpy as np

def mean_distance(x, y):
    nx = len(x)
    result = 0.0
    count = 0
    for i in range(nx):
        result += x[i] - y[i]
        count += 1
    return result / count    

In [26]:
# 이 함수는 무척 느리다.

In [27]:
x = np.random.randn(10000000)

In [28]:
y = np.random.randn(10000000)

In [29]:
%timeit mean_distance(x, y)

3.8 s ± 61.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [30]:
%timeit (x - y).mean()

40 ms ± 3.13 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [31]:
# NumPy로 작성된 코드는 100배 넘게 빠르다.
# 이제 이 함수를 numba.jit 함수를 이용하여 컴파일된 Numba 함수로 바꿔보자.

In [32]:
import numpy as np
import numba as nb

In [33]:
numba_mean_distance = nb.jit(mean_distance)

In [34]:
# 장식자를 사용할 수도 있다.

In [35]:
@nb.jit
def mean_distance(x, y):
    nx = len(x)
    result = 0.0
    count = 0
    for i in range(nx):
        result += x[i] - y[i]
        count += 1
    return result / count

In [36]:
# 이렇게 만들어진 함수는 NumPy 버전보다 조금 더 빠르게 동작한다.

In [37]:
%timeit numba_mean_distance(x, y)

13 ms ± 382 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [38]:
# Numba를 이용해서 모든 파이썬 코드를 컴파일할 수는 없지만 순수 파이썬 코드의 많은 부분을 지원하며 산술 알고리즘을 작성할 경우 특히 유용하다.

In [39]:
# Numba는 심오한 라이브러리로, 다양한 하드웨어를 지원하며 컴파일 모드와 사용자 확장을 지원한다.
# 명시적인 for 문에 기대지 않고 상당수의 NumPy 파이썬 API를 컴파일할 수 있다.
# Numba는 컴파일할 수 없는 함수는 CPython API로 대체하는 동시에 기계코드로 컴파일이 가능한 함수를 구별할 수 있다.
# Numba의 jit 함수는 nopython=True 옵션으로 파이썬 C API를 호출하지 않음으로써 LLVM으로 컴파일될 파이썬 코드를 제한할 수 있다.
# jit(nopython=True)는 numba.njit으로 줄여 쓸 수 있다.

# 앞의 예제는 다음과 같이 쓸 수 있다.

In [41]:
from numba import float64, njit

@njit(float64(float64[:], float64[:]))
def mean_distance(x, y):
    return (x - y).mean()

In [42]:
# Numba 공식 문서를 더 읽어보길 추천한다. 이어서 사용자 정의 NumPy ufunc를 작성하는 방법을 살펴보겠다.

    - A.7.1 Numba를 이용한 사용자 정의 numpy.ufunc 만들기

In [43]:
# numba.vectorize 함수는 내장 ufunc처럼 작동하는 컴파일된 NumPy ufunc를 생성한다.
# numpy.add의 파이썬 구현을 살펴보자.

In [44]:
from numba import vectorize

@vectorize
def nb_add(x, y):
    return x + y

In [45]:
# 이 함수는 아래처럼 사용할 수 있다.

In [46]:
x = np.arange(10)

In [52]:
nb_add(x, x)

array([ 0,  2,  4,  6,  8, 10, 12, 14, 16, 18], dtype=int64)